<img src = "https://drive.google.com/uc?export=view&id=1RQ9YosT6N8bIrbPkehL8hGGY9P1GfHuC" alt = "Encabezado MLDS" width = "100%">  </img>

#**Quiz 1 : Conceptos de SQL con _SQLite_**
---

Este es un quiz para practicar las habilidades adquiridas usando *Python* en el manejo de la librería _SQLite_. En este quiz usted deberá realizar algunas operaciones de consulta de bases de datos con la librería **`sqlite3`** de la librería estándar de _Python_ para una base de datos real con información histórica del campeonato de _Fórmula 1_. 

> **Nota:** Esta tarea va a ser calificada en la plataforma **[UNCode](https://juezun.github.io/)**. Para esto, en cada ejercicio se indicará si es calificable o no, también los lugares donde debe escribir su código sin modificar lo demás con un aproximado de cantidad de líneas a escribir. No se preocupe si su código toma más líneas, esto es simplemente un aproximado destinado a que pueda replantear su estrategia si el código está tomando más de las esperadas. No es un requisito estricto y soluciones más largas también son válidas. Al finalizar, para realizar el envío (*submission*), descargue el notebook como un archivo **`.ipynb`** y haga su entrega a través de **Moodle**.

Ejecute la siguiente celda para importar las librerías.

In [1]:
# Importar librerías
import sqlite3
import numpy as np
import pandas as pd

In [2]:
#TEST_CELL
!python --version

import platform
print('SQLite (Python):', platform.python_version())
print('NumPy', np.__version__)
print('Pandas', pd.__version__)

Python 3.7.12
SQLite (Python): 3.7.12
NumPy 1.19.5
Pandas 1.1.5


Este material fue realizado con las siguientes versiones:

- Python: 3.7.10
- SQLite (Python): 3.7.10
- NumPy: 1.19.5
- Pandas: 1.1.5




## **0.  Base de datos**
---

En esta tarea se va a utilizar el conjunto de datos [**Formula 1 Race Data**](https://www.kaggle.com/cjgdev/formula-1-race-data-19502017) de _Kaggle_, y en particular, una [versión](https://www.kaggle.com/davidcochran/formula-1-race-data-sqlite) del conjunto de datos construido para su uso en el formato del estándar _SQLite_. Este fue recopilado de la plataforma [**Ergast**](http://ergast.com/mrd/) y contiene información de la competición entre los años $1950$ y $2017$ y está compuesto por varias tablas para los conductores, circuitos, fabricantes y carreras, entre otros.

Para cargar el conjunto de datos realizaremos una conexión a un archivo local con extensión **`.sqlite`**. Ejecute la siguiente celda para descargar el archivo:


> **Nota**: El archivo **`Formula1.sqlite`** estará en el ambiente de _UNCode_, no se preocupe por ello.




In [3]:
#TEST_CELL
!wget https://github.com/JuezUN/datasets/raw/master/Formula1.sqlite

--2021-11-13 02:20:52--  https://github.com/JuezUN/datasets/raw/master/Formula1.sqlite
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JuezUN/datasets/master/Formula1.sqlite [following]
--2021-11-13 02:20:52--  https://raw.githubusercontent.com/JuezUN/datasets/master/Formula1.sqlite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1335296 (1.3M) [application/octet-stream]
Saving to: ‘Formula1.sqlite’

Formula1.sqlite     100%[===================>]   1.27M  --.-KB/s    in 0.07s   

2021-11-13 02:20:54 (19.3 MB/s) - ‘Formula1.sqlite’ saved [1335296/1335296]



Ahora, iniciamos la conexión:

In [4]:
#TEST_CELL
# Iniciamos y configuramos la conexión con la 
# base de datos en modo de solo consulta.

connection = sqlite3.connect("Formula1.sqlite")
connection.execute('PRAGMA query_only = ON') 
connection.commit()

En este caso, trabajaremos con 4 de las tablas disponibles. Estas son la tabla de conductores (**`drivers`**), circuitos (**`circuits`**), carreras (**`races`**) y resultados (**`results`**). Veamos los campos de cada una de ellas:

* **Tabla de conductores.**

In [6]:
#TEST_CELL
# Campos de la tabla de conductores.
connection.cursor().execute("PRAGMA table_info(drivers);").fetchall()

[(0, 'driverId', 'INTEGER', 0, None, 1),
 (1, 'driverRef', 'VARCHAR', 0, None, 0),
 (2, 'number', 'VARCHAR', 0, None, 0),
 (3, 'code', 'VARCHAR', 0, None, 0),
 (4, 'forename', 'VARCHAR', 0, None, 0),
 (5, 'surname', 'VARCHAR', 0, None, 0),
 (6, 'dob', 'VARCHAR', 0, None, 0),
 (7, 'nationality', 'VARCHAR', 0, None, 0),
 (8, 'url', 'VARCHAR', 0, None, 0)]

* **Tabla de circuitos.**


In [7]:
#TEST_CELL
# Campos de la tabla de circuitos.
connection.cursor().execute("PRAGMA table_info(circuits);").fetchall()

[(0, 'circuitId', 'INTEGER', 0, None, 1),
 (1, 'circuitRef', 'VARCHAR', 0, None, 0),
 (2, 'name', 'VARCHAR', 0, None, 0),
 (3, 'location', 'VARCHAR', 0, None, 0),
 (4, 'country', 'VARCHAR', 0, None, 0),
 (5, 'lat', 'DECIMAL (8, 6)', 0, None, 0),
 (6, 'lng', 'DECIMAL (9, 6)', 0, None, 0),
 (7, 'alt', 'INTEGER', 0, None, 0),
 (8, 'url', 'VARCHAR', 0, None, 0)]

* **Tabla de carreras.**

In [8]:
#TEST_CELL
# Campos de la tabla de carreras.
connection.cursor().execute("PRAGMA table_info(races);").fetchall()

[(0, 'raceId', 'INTEGER', 0, None, 1),
 (1, 'year', 'INTEGER', 0, None, 0),
 (2, 'round', 'INTEGER', 0, None, 0),
 (3, 'circuitId', 'INTEGER', 0, None, 0),
 (4, 'name', 'VARCHAR', 0, None, 0),
 (5, 'date', 'DATE', 0, None, 0),
 (6, 'time', 'TIME', 0, None, 0),
 (7, 'url', 'VARCHAR', 0, None, 0)]

* **Tabla de resultados**

In [9]:
#TEST_CELL
# Campos de la tabla de resultados.
connection.cursor().execute("PRAGMA table_info(results);").fetchall()

[(0, 'resultId', '', 0, None, 0),
 (1, 'raceId', 'INTEGER', 0, None, 0),
 (2, 'driverId', 'INTEGER', 0, None, 0),
 (3, 'constructorId', 'INTEGER', 0, None, 0),
 (4, 'number', 'INTEGER', 0, None, 0),
 (5, 'grid', 'INTEGER', 0, None, 0),
 (6, 'position', 'INTEGER', 0, None, 0),
 (7, 'positionText', 'VARCHAR', 0, None, 0),
 (8, 'positionOrder', 'INTEGER', 0, None, 0),
 (9, 'points', 'INTEGER', 0, None, 0),
 (10, 'laps', 'INTEGER', 0, None, 0),
 (11, 'time', 'VARCHAR', 0, None, 0),
 (12, 'milliseconds', 'INTEGER', 0, None, 0),
 (13, 'fastestLap', 'VARCHAR', 0, None, 0),
 (14, 'rank', 'INTEGER', 0, None, 0),
 (15, 'fastestLapTime', 'VARCHAR', 0, None, 0),
 (16, 'fastestLapSpeed', 'DECIMAL', 0, None, 0),
 (17, 'statusId', 'INTEGER', 0, None, 0)]

## **1. Circuitos del hemisferio sur.**
---
La primera tabla que consideraremos es la que contiene la lista de las pistas o circuitos (**`circuits`**) de la competición. En esta, se almacena información geográfica del circuito, como el país, ciudad y coordenadas geográficas de su ubicación exacta.

> Las [coordenadas geográficas](https://es.wikipedia.org/wiki/Coordenadas_geogr%C3%A1ficas) permiten codificar la posición en el mundo de un punto determinado y están compuestas por los valores de **latitud** y **longitud**. La latitud representa el ángulo de la ubicación con respecto al plano del ecuador (con $-90°$ para el polo sur y $90°$ para el polo norte) y la longitud representa el ángulo con respecto al meridiano de _Greenwich_ (con valores de  $-180°$ hacia el occidente y $180°$ hacia el oriente).

En este ejercicio usted deberá completar la función **`south_circuits`**, que retorne una cadena de texto con una **consulta SQL válida** que retorne el país, ciudad y nombre de los circuitos ubicados en el **hemisferio sur**.

**Salida:**

* **`query`**: cadena de texto con una consulta válida de _SQLite_. 

In [10]:
# FUNCIÓN CALIFICADA south_circuits:

def south_circuits():
  """
  Retorna:
    query: cadena de texto con una consulta válida de SQLite.
  """  
  ### ESCRIBA SU CÓDIGO AQUÍ ### (~1-8 líneas de código (cadena de texto multilínea))
  # Modifique la siguiente cadena de texto con la sintaxis
  # necesaria para realizar la consulta indicada.
  
  return """\
  SELECT country, location, name
  FROM circuits
  WHERE lat<0
  """

Use la siguiente celda para probar su consulta:

In [11]:
#TEST_CELL
from IPython.display import display
try:
  display(pd.read_sql(south_circuits(), connection))
except Exception as e:
  print("La consulta retornada no es válida. Revise su solución y guíse del siguiente error:")
  print(e)

,country,location,name
0,Australia,Melbourne,Albert Park Grand Prix Circuit
1,Brazil,SÃ£o Paulo,AutÃ_dromo JosÃ© Carlos Pace
2,Argentina,Buenos Aires,AutÃ_dromo Juan y Oscar GÃ¡lvez
3,Australia,Adelaide,Adelaide Street Circuit
4,South Africa,Midrand,Kyalami
5,Brazil,Rio de Janeiro,AutÃ_dromo Internacional Nelson Piquet
6,South Africa,Eastern Cape Province,Prince George Circuit


**Salida esperada:**

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>country</th>
      <th>location</th>
      <th>name</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Australia</td>
      <td>Melbourne</td>
      <td>Albert Park Grand Prix Circuit</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Brazil</td>
      <td>SÃ£o Paulo</td>
      <td>AutÃ_dromo JosÃ© Carlos Pace</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Argentina</td>
      <td>Buenos Aires</td>
      <td>AutÃ_dromo Juan y Oscar GÃ¡lvez</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Australia</td>
      <td>Adelaide</td>
      <td>Adelaide Street Circuit</td>
    </tr>
    <tr>
      <th>4</th>
      <td>South Africa</td>
      <td>Midrand</td>
      <td>Kyalami</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Brazil</td>
      <td>Rio de Janeiro</td>
      <td>AutÃ_dromo Internacional Nelson Piquet</td>
    </tr>
    <tr>
      <th>6</th>
      <td>South Africa</td>
      <td>Eastern Cape Province</td>
      <td>Prince George Circuit</td>
    </tr>
  </tbody>
</table>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Verifique los campos seleccionados con su consulta y asegúrese de indicarlas en minúsculas. Estas deben ser, en ese orden:
  * **`country`:** país.
  * **`location`:** ciudad.
  * **`name`:** nombre del circuito.



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Los puntos ubicados en el hemisferio sur son aquellos con valores negativos de **latitud**. Utilice una expresión **`WHERE`** para condicionar la consulta a los valores negativos del campo **`lat`** de la tabla.


## **2. Pilotos con la misma nacionalidad.**
---
Otra de las tablas disponibles en la base de datos es la tabla de los pilotos (**`drivers`**) con información básica como sus códigos de identificación, nombres, apellidos, fecha de nacimiento, nacionalidad y una url de *Wikipedia* con más información.

> Para conocer las nacionalidades únicas de la base de datos, podemos utilizar la palabra clave **`DISTINCT`**, que permite obtener los valores sin repeticiones de un campo determinado.


In [12]:
#TEST_CELL
# Nacionalidades únicas del campo 'nationality'.
pd.read_sql("SELECT DISTINCT nationality FROM drivers", connection).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
nationality,British,German,Spanish,Finnish,Japanese,French,Polish,Brazilian,Italian,Australian,Austrian,American,Dutch,Colombian,Portuguese,Canadian,Indian,Hungarian,Irish,Danish,Argentine,Czech,Malaysian,Swiss,Belgian,Monegasque,Swedish,Venezuelan,New Zealander,Chilean,Mexican,South African,Liechtensteiner,Rhodesian,American-Italian,Uruguayan,Argentine-Italian,Thai,East German,Russian,Indonesian


En este ejercicio usted deberá completar la función **`drivers_by_nation`**, que reciba una cadena de texto con una nacionalidad (correspondiente a valores del campo **`nationality`**) y retorne una cadena de texto con una **consulta SQL válida** que permita obtener el nombre, apellido y url de los pilotos de dicha nacionalidad.

**Entrada:**

* **`country`**: cadena de texto con la nacionalidad de los conductores que se desean obtener, que corresponde a alguno de los valores de la tabla anterior. 

**Salida:**

* **`query`**: cadena de texto con una consulta válida de _SQLite_. 

In [27]:
# FUNCIÓN CALIFICADA drivers_by_nation:

def drivers_by_nation(country):
  """
  Entradas:
    country: cadena de texto con la nacionalidad de los conductores. 
  Retorna:
    query: cadena de texto con una consulta válida de SQLite.
  """  
  ### ESCRIBA SU CÓDIGO AQUÍ ### (~1-8 líneas de código (cadena de texto multilínea))
  # Modifique la siguiente cadena de texto con la sintaxis
  # necesaria para realizar la consulta indicada.
  
  return """\
  SELECT forename, surname, url  
  FROM drivers
  WHERE nationality = '%s'
  """ % country 

Use la siguiente celda para probar su código:

In [28]:
#TEST_CELL
from IPython.display import display
try:
  display(pd.read_sql(drivers_by_nation('Colombian'), connection))
except Exception as e:
  print("La consulta retornada no es válida. Revise su solución y guíese del siguiente error:")
  print(e)

,forename,surname,url
0,Juan,Pablo Montoya,http://en.wikipedia.org/wiki/Juan_Pablo_Montoya
1,Roberto,Guerrero,http://en.wikipedia.org/wiki/Roberto_Guerrero
2,Ricardo,Londo̱o,http://en.wikipedia.org/wiki/Ricardo_Londo%C3%B1o


**Salida esperada:**

|    | forename   | surname       | url                                               |
|---:|:-----------|:--------------|:--------------------------------------------------|
|  0 | Juan       | Pablo Montoya | http://en.wikipedia.org/wiki/Juan_Pablo_Montoya   |
|  1 | Roberto    | Guerrero      | http://en.wikipedia.org/wiki/Roberto_Guerrero     |
|  2 | Ricardo    | Londo̱o        | http://en.wikipedia.org/wiki/Ricardo_Londo%C3%B1o |

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Verifique los campos seleccionados con su consulta y asegúrese de indicarlas en minúsculas. Estas deben ser, en ese orden:
  * **`forename`:** nombres.
  * **`surname`:** apellidos.
  * **`url`:** url de _Wikipedia_.



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Utilice una expresión **`WHERE`** para condicionar la consulta a los valores exactos del campo **`nationality`**. Utilice una cadena de texto [**f-string**](https://realpython.com/python-f-strings/) o métodos similares como la concatenación de cadenas de texto para usar el valor dado como argumento. 

Es importante que **tenga en cuenta el uso de comillas**. De lo contrario se interpretará el valor como el nombre de una variable, en vez de una cadena de texto. La cadena de su respuesta debería quedar:

```sql
...
WHERE 
  _______ = 'Colombian'
...
```


Si por el contrario queda como se muestra a continuación no se obtendrá un resultado correcto pues el motor intentará encontrar la variable **`Colombian`**.

```sql
...
WHERE 
  _______ = Colombian
...
```




## **3. Tabla de posiciones de una carrera**
---
Finalmente, la base de datos cuenta con una tabla para las carreras (**`races`**) y otra para los resultados individuales de un corredor en una carrera determinada (**`results`**). Esta última referencia a los índices de las otras $3$ tablas (en loscampos **`driverId`**, **`circuitId`** y **`raceId`**), que contienen la información del corredor, el circuito y la carrera donde se produjo este resultado. Esta estrategia permite optimizar las consultas y el espacio de almacenamiento al no almacenar información redundante o innecesaria, como la fecha de nacimiento de un corredor, y que se puede obtener con la unión de las tablas de la base de datos.

En este ejercicio usted deberá completar la función **`race_standings`**, que reciba el código de una carrera (**`race_id`**) y retorne la tabla de posiciones de la carrera, con la posición, el nombre, apellido y nacionalidad del corredor. 

Además, se espera que retorne los resultados **ordenados por posición**. Para esto, deberá utilizar el modificador **`ORDER BY`**, que se usa de la siguiente forma:

```sql
SELECT ...
FROM ...
ORDER BY variable
```

De esta manera el motor realizará el ordenamiento y retornará los valores en orden numérico o alfanumérico con respecto a la variable (o variables) indicadas. Consulte el [siguiente enlace](https://www.sqlitetutorial.net/sqlite-order-by/) para más información.

Este modificador puede usarse en conjunto con modificadores como **`WHERE`** y expresiones **`JOIN`**. 

> **Nota:** Se espera que su solución utilice expresiones condicionales con **`WHERE`** y una unión entre tablas con **`JOIN`**. 

**Entrada:**

* **`raceId`**: cadena de texto con el ID de la carrera a la de los conductores que se desean obtener, que corresponde al índice de la tabla **`races`**. 

**Salida:**

* **`query`**: cadena de texto con una consulta válida de _SQLite_. 


In [57]:
# FUNCIÓN CALIFICADA race_standings:

def race_standings(raceId):
  """
  Entradas:
    raceId: código de la carrera a retornar. 
  Retorna:
    query: cadena de texto con una consulta válida de SQLite.
  """    
  ### ESCRIBA SU CÓDIGO AQUÍ ### (~1-12 líneas de código (cadena de texto multilínea))
  # Modifique la siguiente cadena de texto con la sintaxis
  # necesaria para realizar la consulta indicada.

  return f"""\
  SELECT position, forename, surname, nationality
  FROM results 
  JOIN races 
  ON (results.raceId=races.raceId)
  JOIN drivers 
  ON (results.driverId=drivers.driverId)
  WHERE results.raceId=%i
  ORDER BY position
  """ % raceId

Use la siguiente celda para probar su código:

In [58]:
#TEST_CELL
from IPython.display import display
try:
  display(pd.read_sql(race_standings(250), connection, index_col = 'position').head())
except Exception as e:
  print("La consulta retornada no es válida. Revise su solución y guíese del siguiente error:")
  print(e)

,forename,surname,nationality
position,,,
1,Michael,Schumacher,German
2,Damon,Hill,British
3,Martin,Brundle,British
4,Heinz-Harald,Frentzen,German
5,Mark,Blundell,British


**Salida esperada:**

|   position | forename     | surname    | nationality   |
|-----------:|:-------------|:-----------|:--------------|
|          1 | Michael      | Schumacher | German        |
|          2 | Damon        | Hill       | British       |
|          3 | Martin       | Brundle    | British       |
|          4 | Heinz-Harald | Frentzen   | German        |
|          5 | Mark         | Blundell   | British       |

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Verifique los campos seleccionados con su consulta y asegúrese de indicarlas en minúsculas. Estas deben ser, en ese orden:

  * **`position`:** posición de llegada a la meta.
  * **`forename`:** nombres.
  * **`surname`:** apellidos.
  * **`nationality`:** nacionalidad.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Utilice una expresión **`WHERE`** para condicionar la consulta a los valores exactos del campo **`nationality`**. Utilice una cadena de texto [**f-string**](https://realpython.com/python-f-strings/) o métodos similares como la concatenación de cadenas de texto para usar el valor dado como argumento. 

En este caso el valor es un número entero, por lo que debería realizar la expresión **sin comillas**:

  ```sql
...
WHERE 
  _______ = 125
...
```

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 3</b></font>
</summary>

* Para obtener la información de los corredores deberá realizar un **`JOIN`** entre la tabla de resultados (**`results`**) y la de corredores (**`drivers`**). Para esto, utilice las ids de los corredores, correspondiente al campo **`driverId`** de ambas tablas.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 4</b></font>
</summary>

* Utilice el campo de posición **`position`** para ordenar los resultados de la consulta. Al adicionar la expresión **`ORDER BY`** la consulta final podría tener la siguiente estructura:

  ```sql
SELECT 
  var_a, var_b, ...
FROM 
  table_a JOIN table_b
ON 
  table_a.var_id_a = table_b.var_id_b
WHERE
  var_c = value_c
ORDER BY
  var_order;
```

**¡Felicitaciones!** Ha terminado la tarea de conceptos de SQL de la Unidad 1. ¡Buen trabajo!

## **Entrega**

Para entregar el notebook por favor haga lo siguiente:
1. Descargue el notebook (`Archivo` -> `Descargar .ipynb`).
2. Ingrese a Moodle.
3. Realice el envío del *notebook* que descargó en la tarea (o quiz) correspondiente.
4. Recuerde que si tiene algún error, puede hacer múltiples intentos de envío en UNCode. 

## **Créditos**
---

* **Profesor:** [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)
* **Asistentes docentes:**
  - Alberto Nicolai Romero Martínez  

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*